In [1]:
import pandas as pd
from pandas import DataFrame
import requests
import os
import os.path
import datetime as dt
import json
import numpy as np
from datetime import timedelta, datetime
from dateutil import parser
import math
%matplotlib inline
from matplotlib import pyplot as plt

from pathlib import Path
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima_model import ARMA

from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

Python-dotenv could not parse statement starting at line 20
Python-dotenv could not parse statement starting at line 21


In [2]:
from kucoin.client import Client

api_key = '5f4a5b8e4128080006df7c63'
api_secret = 'c610ac4c-2754-4d5c-b39b-a3b055558413'
api_passphrase = 'STAGE001'

In [3]:
client = Client(api_key, api_secret, api_passphrase, sandbox=True)

In [4]:
# Load environment variables 
binance_api_key = os.getenv("GkFIyM2AydVGyMCNpp4snFLruNyWjlu4tPFXtzdwO5Ut7J0Gc1IactPdb6ZlrJXG")
binance_secret_key= os.getenv("Qg5PWRdkzsHP2KiLOPk3xsPDSRNmpukdIuGYjDrDkEllX1wCTfnNzDG9in4RXExn")


In [5]:
type(binance_api_key)

NoneType

In [6]:
type(binance_secret_key)

NoneType

In [7]:
# Set up client 
from binance.client import Client
binance_client = Client(api_key="GkFIyM2AydVGyMCNpp4snFLruNyWjlu4tPFXtzdwO5Ut7J0Gc1IactPdb6ZlrJXG", api_secret="Qg5PWRdkzsHP2KiLOPk3xsPDSRNmpukdIuGYjDrDkEllX1wCTfnNzDG9in4RXExn")

# Constants
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750

# Create collecting new data function
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('29 Aug 2020', '%d %b %Y')#<-- Can change start date here
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    return old, new

# Create function to retreive tickers and timestamps
def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2015', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [8]:
symbols = ['ETHUSDT', 'LINKUSDT','LTCUSDT', 'EOSUSDT', 'BTCUSDT']

In [9]:
price_data = []
for symbol in symbols:
    prices=get_all_binance(symbol, kline_size='1m')
    prices=prices.loc[:,['close']]
    price_data.append(prices.assign(coin=symbol))

coins = pd.concat(price_data)
coins

All caught up..!
All caught up..!
All caught up..!
All caught up..!
All caught up..!


,close,coin
timestamp,,
2020-08-29 00:00:00,395.50000000,ETHUSDT
2020-08-29 00:01:00,395.35000000,ETHUSDT
2020-08-29 00:02:00,395.61000000,ETHUSDT
2020-08-29 00:03:00,395.45000000,ETHUSDT
2020-08-29 00:04:00,395.01000000,ETHUSDT
2020-08-29 00:05:00,394.83000000,ETHUSDT
2020-08-29 00:06:00,394.68000000,ETHUSDT
2020-08-29 00:07:00,394.48000000,ETHUSDT
2020-08-29 00:08:00,394.60000000,ETHUSDT


In [10]:
coins.index.name = None 

In [11]:
coins.head()

,close,coin
2020-08-29 00:00:00,395.50000000,ETHUSDT
2020-08-29 00:01:00,395.35000000,ETHUSDT
2020-08-29 00:02:00,395.61000000,ETHUSDT
2020-08-29 00:03:00,395.45000000,ETHUSDT
2020-08-29 00:04:00,395.01000000,ETHUSDT


In [12]:
coins = coins.pivot(columns='coin')

In [13]:
coins.head()

close                                         \
coin                        BTCUSDT     EOSUSDT       ETHUSDT     LINKUSDT   
2020-08-29 00:00:00  11534.32000000  3.11810000  395.50000000  15.13330000   
2020-08-29 00:01:00  11529.92000000  3.11690000  395.35000000  15.13060000   
2020-08-29 00:02:00  11535.53000000  3.12160000  395.61000000  15.17750000   
2020-08-29 00:03:00  11531.24000000  3.12050000  395.45000000  15.16080000   
2020-08-29 00:04:00  11524.88000000  3.11970000  395.01000000  15.15100000   

                                  
coin                     LTCUSDT  
2020-08-29 00:00:00  57.41000000  
2020-08-29 00:01:00  57.39000000  
2020-08-29 00:02:00  57.53000000  
2020-08-29 00:03:00  57.56000000  
2020-08-29 00:04:00  57.50000000

In [14]:
coins.columns = ['BTCUSDT', 'EOSUSDT','ETHUSDT','LINKUSDT','LTCUSDT']

In [15]:
coins.dtypes

BTCUSDT     object
EOSUSDT     object
ETHUSDT     object
LINKUSDT    object
LTCUSDT     object
dtype: object

In [16]:
coins=coins.apply(pd.to_numeric)

In [17]:
df = coins

In [18]:
df.head()

,BTCUSDT,EOSUSDT,ETHUSDT,LINKUSDT,LTCUSDT
2020-08-29 00:00:00,11534.32,3.1181,395.50,15.1333,57.41
2020-08-29 00:01:00,11529.92,3.1169,395.35,15.1306,57.39
2020-08-29 00:02:00,11535.53,3.1216,395.61,15.1775,57.53
2020-08-29 00:03:00,11531.24,3.1205,395.45,15.1608,57.56
2020-08-29 00:04:00,11524.88,3.1197,395.01,15.1510,57.50


In [19]:
df.tail()

,BTCUSDT,EOSUSDT,ETHUSDT,LINKUSDT,LTCUSDT
2020-08-31 21:46:00,11679.63,3.2196,432.99,16.1366,61.07
2020-08-31 21:47:00,11665.47,3.2183,432.58,16.0997,60.94
2020-08-31 21:48:00,11663.61,3.2165,432.67,16.1565,60.90
2020-08-31 21:49:00,11665.06,3.2181,432.70,16.1506,60.90
2020-08-31 21:50:00,11662.80,3.2179,432.68,16.1298,60.89


In [20]:
independent = 'ETHUSDT'
dependent = 'BTCUSDT'

In [21]:
def generate_signals(df,independent,dependent,multiplier):
    df = df.pct_change()
    df = df.dropna()
   
    X = df[independent].values.reshape(-1, 1)
    y = df[dependent]
    model = LinearRegression()
    model.fit(X, y)
    predicted_y_values = model.predict(X)
    residuals = df.loc[:,'BTCUSDT'] - predicted_y_values
    model_AR = ARMA(residuals.values, order = (1,0))
    results = model_AR.fit()
    a = results.params[0]
    b = results.params[1]
    kappa = -np.log(b)
    m = a / (1 - np.exp(-kappa))
    residuals_shifted = residuals.shift()
    residuals2 = residuals - (a + b*residuals_shifted)
   
    sigma_epsilon = residuals2.std()
    
    if residuals2[-1] > multiplier*sigma_epsilon:
        return -1.0
    elif residuals2[-1] < -multiplier*sigma_epsilon:
        return 1.0
    else:
        return 0.0

In [ ]:
import time

while True:
    print("Starting STAGE")

    symbols = ['ETHUSDT','BTCUSDT']

    price_data = []
    for symbol in symbols:
        prices=get_all_binance(symbol, kline_size='1m')
        prices=prices.loc[:,['close']]
        price_data.append(prices.assign(coin=symbol))

    coins = pd.concat(price_data)
    
    coins.index.name = None 

    coins = coins.pivot(columns='coin')

    coins=coins.apply(pd.to_numeric)
   
    coins.columns = ['BTCUSDT','ETHUSDT']
    
    independent = 'ETHUSDT'
    dependent = 'BTCUSDT'
    
    position = generate_signals(df,independent,dependent,1.0)
    
    if position == 1:
        client.create_market_order('BTC-USDT', Client.SIDE_BUY, funds=200)
        client.create_market_order('ETH-USDT', Client.SIDE_SELL, funds=200)
        print('sell dependent, buy independent')
        time.sleep(55)
        client.create_market_order('BTC-USDT', Client.SIDE_SELL, funds=200)
        client.create_market_order('ETH-USDT', Client.SIDE_BUY, funds=200)
        print('closing position')
    elif position == -1:
        client.create_market_order('BTC-USDT', Client.SIDE_SELL, funds=200)
        client.create_market_order('ETH-USDT', Client.SIDE_BUY, funds=200)
        print('buy dependent, sell independent')
        time.sleep(55)
        client.create_market_order('ETH-USDT', Client.SIDE_SELL, funds=200)
        client.create_market_order('BTC-USDT', Client.SIDE_BUY, funds=200)
        print('closing position')
        
    else:
        print('hold')
        time.sleep(55)
    
        
    time.sleep(5)
    

Starting STAGE
All caught up..!
All caught up..!
hold
Starting STAGE
All caught up..!
All caught up..!
hold
Starting STAGE
All caught up..!
All caught up..!
hold


In [ ]:
client.create_market_order('ETH-BTC', Client.SIDE_BUY, size=30)

In [ ]:
accounts = client.get_accounts()

In [ ]:
accounts

In [ ]:
currencies = client.get_currencies()

In [ ]:
klines = client.get_kline_data('ETH-USDT')

In [ ]:
currencies = client.get_currencies()

In [ ]:
currency = client.get_currency('BTC')

In [ ]:
accounts = client.get_accounts()

In [ ]:
accounts

In [ ]:
account = client.create_account('trade', 'BTC')

In [ ]:
order = client.create_market_order('BTC-USDT', Client.SIDE_BUY, size=0.5)

In [ ]:
order